# Регуляризация Тихонова для решения СЛАУ

Рассмотрим СЛАУ:

$$
Ax=b,\quad z\in R^n
$$



Регуляризация Тихонова заключается в минимизации сглаживающего функционала:

\begin{equation*}
  \| A {x} - {b} \|_2^2 + \alpha \| {x} \|_2^2 \to \min.
\end{equation*}

Известно, что от задачи поиска минимума такого функционала можно перейти к задаче поиска решения уравнения 

\begin{equation*}
\newcommand{\trans}[1]{{#1}^{\text{T}}}      % transposition
\newcommand{\vec}[1]{{#1}}
\newcommand{\invtrans}[1]{{#1}^{-\text{T}}}  % inverse and transposition
  (\trans{A}A + \alpha E) \vec{x}_\alpha = \trans{A} \vec{b},
\end{equation*}

где $E$ - единичная матрица, совпадающая по размерности с $A$, $\alpha$ - параметр регуляризации, задающий "сглаженность" решения.

Решение:

\begin{equation*}
   \vec{x}_\alpha=(\trans{A}A + \alpha E)^{-1}\trans{A} \vec{b},
\end{equation*}

При $\alpha\to0$ решение такого уравнения сводится к изначальной задаче.


Реализация регуляризации возможна как учетом вектора ошибки (если он известен), так и без него. 

Задача выбора параметра регуляризации может решаться разными способами, наипростейший (но неустойчивый) способ - минимизация невязки псевдорешения, более устойчивый - метод обобщенной невязки.

В этой программе для выбора параметра применяется т.н. "обобщенная перекрёстная проверка".

### Обобщенная перекрёстная проверка

Не приводя вывод, укажем, что определение оптимального $\alpha$, в случае когда нет априорной информации об ошибке, может быть выполнено методом перекрестной проверки (статистический метод). [(см. статью, формула 9)](https://pdfs.semanticscholar.org/886c/dafdfec18b0fdaa2a603d52126e212a144aa.pdf)

В этом методе составляется функционал перекрестной проверки $\Phi$, и для нахождения оптимального $\alpha$ его необходимо минимизировать:

\begin{align*}
\DeclareMathOperator{\trace}{trace}
  \Phi (\alpha) := {} &
    \frac{\| \vec{b} - A (\trans{A} A + \alpha I)^{-1}
                       \trans{A} \vec{b} \|}
         {\trace \bigl( I - A (\trans{A} A + \alpha I)^{-1}
                            \trans{A} \bigr)}                         \\
  {} = {} &
    \frac{\| (A \trans{A} + \alpha I)^{-1} \vec{b} \|}
         {\trace \bigl( (A \trans{A} + \alpha I)^{-1} \bigr)} \to \min.
\end{align*}

### Модельная задача - возмущение СЛАУ

Рассмотрим СЛАУ с малым возмущением $\varepsilon$:

\begin{equation}
\begin{pmatrix} 
  1     & 1\\ 
  0 & \varepsilon
\end{pmatrix}\cdot\begin{pmatrix} 
  x_1\\
  x_2\\
\end{pmatrix}=
\begin{pmatrix} 
    2\\
    \varepsilon^2
\end{pmatrix}
\label{eq:1} \tag{1}
\end{equation}

Здесь легко записывается невозмущенная система (в силу того, что мы сами задали возмущение):

$$
\begin{pmatrix} 
  1     & 1\\ 
  0 & 0
\end{pmatrix}\cdot\begin{pmatrix} 
  x_1\\
  x_2\\
\end{pmatrix}=
\begin{pmatrix} 
    2\\
    0
\end{pmatrix}
\label{eq:2} \tag{2}
$$

Для невозмущенной системы существует нормальное решение. По определению [(см. книгу, стр 150)](https://www.twirpx.com/file/2488131/) нормальное решение можно получить, взяв возмущенную систему, домножив левую и правую части на $\trans{A}$, затем найти решение и устремить возмущение к нулю:

$$
    x_{normal}=\left[(\trans{A}A)^{-1}\cdot\trans{A}b \right]_{\varepsilon=0}=(1,1)
$$

В то же время, если рассчитать классическое решение возмущенной системы $\eqref{eq:1}$, а потом устремить возмущение в ноль, классическое решение не совпадает с нормальным:

$$
    x_{classic}=\left[A^{-1}\cdot b \right]_{\varepsilon=0}=(2-\varepsilon,\varepsilon)_{\varepsilon=0}=(2,0)
$$

Практически такую систему можно (чисто теоретически, так как пример чисто математический) в результате измерений, получив матрицу $A$ и вектор $b$ с такой погрешностью, что $A_{22}=\varepsilon$ и $b_2=\varepsilon^2$, а решение будет находиться близким к классическому для невозмущенной системы. 

Хотя такое решение будет казаться хорошим (честно рассчитанным), но тем не менее, оно получено при ненулевом возмущении, обеспеченным погрешностью, а значит, реальный ответ должен быть решением невозмущенной системы (которое существует только нормальное).

Таким образом, можно сформулировать критерий проверки нашего алгоритма регуляризации: для больших возмущений он должен давать близкое к классическому решение (большое возмущение не может быть объяснено погрешностью), а для малых должен давать  близкое к псевдорешению решение.

In [40]:
from numpy import trace, array,identity
from numpy.linalg import inv,norm,solve,pinv
from scipy.optimize import minimize

n=2
epsilon=100

# Возмущенное уравнение
A=array([[1,1],
        [0,epsilon]])

b=array([2,
        epsilon**2])

# Единичная матрица размерности n x n
I=identity(n)

# Решение функционала Тихонова
def z(alpha):
    return inv(A.T@A+alpha*I)@(A.T@b)

# Функционал обобщенной перекрестной проверки
def G(alpha):
    return norm((I-A@inv(A.T@A+alpha*I)@A.T)@b)**2/\
                trace(I-A@inv(A.T@A+alpha*I)@A.T)**2

alpha = minimize(G, [-1/2]).x

print('Regular solve',z(alpha))
try:
    print('Classic solve',inv(A)@b)
except:
    print('Pseudo solve',pinv(A)@b)

Regular solve [-94.15682875  99.99920768]
Classic solve [-98. 100.]
